In [1]:
cd /home/daliu/Src/spykshrk_realtime/

/home/daliu/Src/spykshrk_realtime


In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches
import json
import os
import scipy.signal

from spykshrk.realtime.simulator import nspike_data

from spykshrk.franklab.pp_decoder.util import gaussian, normal2D, apply_no_anim_boundary, simplify_pos_pandas
from spykshrk.franklab.pp_decoder.pp_clusterless import OfflinePPDecoder
from spykshrk.franklab.pp_decoder.data_containers import EncodeSettings, DecodeSettings, SpikeObservation, \
                                                         LinearPosition, StimLockout, Posteriors, \
                                                         FlatLinearPosition

from spykshrk.franklab.pp_decoder.visualization import DecodeVisualizer
from spykshrk.franklab.pp_decoder.decode_error import LinearDecodeError

        
%load_ext Cython

%matplotlib inline

#pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.precision', 4)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 15)
#pd.set_option('display.width', 80)

idx = pd.IndexSlice

matplotlib.rcParams.update({'font.size': 28})

%pdb

Automatic pdb calling has been turned ON


In [30]:
%%time
# Load merged rec HDF store based on config

#config_file = '/opt/data36/daliu/realtime/spykshrk/ripple_dec/bond.config.json'
#config_file = '/opt/data36/daliu/realtime/spykshrk/dec_60uv/bond.config.json'
config_file = '/home/daliu/Src/spykshrk_realtime/config/bond_single.json'
config = json.load(open(config_file, 'r'))

day = config['simulator']['nspike_animal_info']['days'][0]
epoch = config['simulator']['nspike_animal_info']['epochs'][0]
time_bin_size = config['pp_decoder']['bin_size']

# Extract just encode and decode settings from config
encode_settings = EncodeSettings(config)
decode_settings = DecodeSettings(config)

# Grab animal linearized real position
nspike_anim = nspike_data.AnimalInfo(**config['simulator']['nspike_animal_info'])
pos = nspike_data.PosMatDataStream(nspike_anim)
pos_data = pos.data

spk = nspike_data.SpkDataStream(nspike_anim)
spk_data = spk.data

# Encapsulate linear position
lin_obj = LinearPosition.from_nspike_posmat(pos_data, encode_settings)

CPU times: user 6.57 s, sys: 970 ms, total: 7.54 s
Wall time: 7.54 s


In [4]:
pos_data

lin_dist_well                          lin_vel                      seg_idx
                                  well_center well_left well_right well_center well_left well_right seg_idx
day epoch timestamp     time                                                                               
4.0 1.0   73,830,339.0  2,461.0          27.8     142.1      144.5         7.5     134.4      136.4     1.0
          73,831,341.0  2,461.0          26.9     143.0      145.4         7.5     134.4      136.4     1.0
          73,832,343.0  2,461.1          25.5     144.3      146.8         6.7     134.5      136.6     1.0
          73,833,342.0  2,461.1          24.6     145.2      147.7         5.8     134.1      136.1     1.0
          73,834,344.0  2,461.1          23.3     146.6      149.0         4.9     133.2      135.1     1.0
...                                       ...       ...        ...         ...       ...        ...     ...
          102,145,374.0 3,404.8           7.0     162.8      165.3        -4.0    -128.6     -130.5     1.0
          102,146,376.0 3,404.9           7.1     162.8      165.2        -4.2    -131.4     -133.4     1.0
          102,147,378.0 3,404.9           7.5     162.3      164.8        -4.4    -133.7     -135.7     1.0
          102,148,377.0 3,404.9           7.5     162.3      164.8        -4.5    -135.4     -137.5     1.0
          102,149,379.0 3,405.0           7.6     162.3      164.7        -4.7    -136.5     -138.6     1.0

[28290 rows x 7 columns]

In [16]:
spk_data.loc[idx[4,1,:],:]

0   1   2   3   4   5   6  ...  33  34  35  36  37  38  39
day epoch tet timestamp channel                             ...                            
4   1     5   73831218  0         7   1  -5  -7  -7  -7  -4 ... -11  -7  -1   3  10  15   9
                        1        18  23  20   9   0  -2   2 ...  -6  -3   2  10  17  15   6
                        2         5   7  11  10  12  17  24 ...  -4   5  13  16  19  18  10
                        3         2   4   3  -2  -2   1   4 ... -18 -16 -10   1  15  21  13
              73831404  0       -10 -11  -8  -5 -10 -16  -5 ...  29  27  21  10  -2 -15 -28
...                              ..  ..  ..  ..  ..  ..  .. ...  ..  ..  ..  ..  ..  ..  ..
          29  102149151 3        21  19  12   7   2   1   7 ...  -4   4   8   9   2 -10 -21
              102149298 0        -3   5  12  10   9  17  33 ...  10  11  11  15  20  23  23
                        1         6  20  24  21  26  40  46 ...  10  12  11  10   9   6   8
                        2         4  11  15  14  18  26  26 ...   2   3   8  18  22  17  10
                        3         5   2  -8 -14  -7  10  19 ...  14  15   8   9  14  14  19

[1637360 rows x 40 columns]

In [22]:
spk_data.sort_index(level='timestamp').values[0:4,:]

array([[-21, -29, -26, -16,  -3,   4,   8,  15,  25,  30,  22,   5,  -8,
         -8,   6,  30,  46,  45,  33,  17,   2, -12, -24, -34, -40, -49,
        -62, -71, -65, -51, -40, -34, -25, -13,  -5,  -6,  -8,  -7,  -3,
          4],
       [-16, -18, -17, -15, -13,  -2,  16,  30,  33,  23,   5, -15, -31,
        -31, -16,   0,  11,  16,  15,  10,   4,  -2,  -8, -12, -14, -18,
        -23, -28, -29, -25, -17, -13, -16, -16,  -9,  -1,   6,  11,  12,
         11],
       [-23, -27, -29, -24, -13,  -1,  11,  30,  51,  51,  24, -11, -34,
        -37, -23,   0,  20,  26,  18,   9,   4,   4,  -2, -13, -25, -35,
        -42, -45, -42, -36, -27, -21, -16,  -9,  -1,   2,   4,   6,   5,
          7],
       [-16, -19, -18,  -7,   9,  19,  18,  14,  10,   9,   4,  -7, -20,
        -25, -19,  -7,   7,  20,  29,  28,  22,  13,   0, -10, -16, -22,
        -29, -36, -37, -33, -30, -29, -29, -23, -13,  -8,  -6,  -5,  -1,
          4]], dtype=int16)

In [52]:
gen = spk()

In [53]:
test = next(gen)    

In [55]:
print(test)

SpikePoint({'timestamp': 73830030, 'ntrode_id': 14, 'data': array([[-21, -29, -26, -16,  -3,   4,   8,  15,  25,  30,  22,   5,  -8,
         -8,   6,  30,  46,  45,  33,  17,   2, -12, -24, -34, -40, -49,
        -62, -71, -65, -51, -40, -34, -25, -13,  -5,  -6,  -8,  -7,  -3,
          4],
       [-16, -18, -17, -15, -13,  -2,  16,  30,  33,  23,   5, -15, -31,
        -31, -16,   0,  11,  16,  15,  10,   4,  -2,  -8, -12, -14, -18,
        -23, -28, -29, -25, -17, -13, -16, -16,  -9,  -1,   6,  11,  12,
         11],
       [-23, -27, -29, -24, -13,  -1,  11,  30,  51,  51,  24, -11, -34,
        -37, -23,   0,  20,  26,  18,   9,   4,   4,  -2, -13, -25, -35,
        -42, -45, -42, -36, -27, -21, -16,  -9,  -1,   2,   4,   6,   5,
          7],
       [-16, -19, -18,  -7,   9,  19,  18,  14,  10,   9,   4,  -7, -20,
        -25, -19,  -7,   7,  20,  29,  28,  22,  13,   0, -10, -16, -22,
        -29, -36, -37, -33, -30, -29, -29, -23, -13,  -8,  -6,  -5,  -1,
          4]], dtype=i

In [42]:
for ii in range(1,5,2):
    print(ii)

1
3
